# T_2 Ramsey Experiment

This experiment serves as one of the series of experiments used to characterize a single qubit. Its purpose is to determine two of the qubit's properties: *Ramsey* or *detuning frequency* and $T_2\ast$. The rough frequency of the qubit was already determined previously. Here, we would like to measure the *detuning*, that is  the difference between the qubit's precise frequency and the frequency of the rotation pulses (based on the rough frequency). This part of the experiment is called a *Ramsey Experiment*. $T_2\ast$ represents the rate of decay toward a mixed state, when the qubit is initialized to the |+⟩ state.

In [1]:
import qiskit
from qiskit_experiments.characterization import T2Ramsey

The circuit used for the experiment comprises the following:

    1. Hadamard gate
    2. delay
    3. p (phase) gate that rotates the qubit in the x-y plane 
    4. Hadamard gate
    5. measurement

During the delay time, we expect the qubit to precess about the z-axis. If the p gate and the precession offset each other perfectly, then the qubit will arrive at the |0⟩ state (after the second Hadamard gate). By varying the extension of the delays, we get a series of oscillations of the qubit state between the |0⟩ and |1⟩ states. We can draw the graph of the resulting function, and can analytically extract the desired values.

In [2]:
# set the computation units to microseconds
unit = 'us' #microseconds
qubit = 0
# set the desired delays
delays = list(range(1, 150, 2))

In [3]:
# Create a T2Ramsey experiment. Print the first circuit as an example
exp1 = T2Ramsey(qubit, delays, unit=unit)
print(exp1.circuits()[0])

     ┌───┐┌──────────────┐┌──────┐ ░ ┌───┐ ░ ┌─┐
q_0: ┤ H ├┤ DELAY(1[us]) ├┤ P(0) ├─░─┤ H ├─░─┤M├
     └───┘└──────────────┘└──────┘ ░ └───┘ ░ └╥┘
c: 1/═════════════════════════════════════════╩═
                                              0 


We run the experiment on a simple, simulated backend, created specifically for this experiment's tutorial.

In [4]:
from qiskit_experiments.test.t2ramsey_backend import T2RamseyBackend
# MockJob is a wrapper for the backend, to give it the form of a job
from qiskit_experiments.test.mock_job import MockJob
import qiskit_experiments.matplotlib
from qiskit_experiments.matplotlib import pyplot, requires_matplotlib
from qiskit_experiments.matplotlib import HAS_MATPLOTLIB

conversion_factor = 1E-6
# The behavior of the backend is determined by the following parameters
backend = T2RamseyBackend(
                    p0={"a_guess":[0.5], "t2ramsey":[80.0], "f_guess":[0.02], "phi_guess":[0.0],
                        "b_guess": [0.5]},
                    initial_prob_plus=[0.0],
                    readout0to1=[0.02],
                    readout1to0=[0.02],
                    conversion_factor=conversion_factor,
                )


The resulting graph will have the form:
$ f(t) = a^{-t/T_2*} \cdot cos(2 \pi f t + \phi) + b $
where *t* is the delay, $T_2*$ is the decay factor, and *f* is the detuning frequency.
`conversion_factor` is a scaling factor that depends on the measurement units used. It is 1E-6 here, because the unit is microseconds.

In [ ]:
exp1.set_analysis_options(user_p0=None, plot=True)
expdata1 = exp1.run(backend=backend, shots=2000)
print(expdata1)


### Providing initial user estimates
The user can provide initial estimates for the parameters to help the analysis process. Because we the curve is expected to decay toward $0.5$, the natural choice for parameters $A$ and $B$ is $0.5$. Varying the value of $\phi$ would shift the graph along the x-axis. Since this is not of interest to us, we can safely initialize $\phi$ to 0. t2ramsey and f are the parameters of interest. Good estimates for them are values computed in previous experiments on this qubit or a computed for other qubits.

In [ ]:
from qiskit_experiments.characterization import T2RamseyAnalysis
user_p0={
    "A": 0.5,
    "t2ramsey": 85.0,
    "f": 0.021,
    "phi": 0,
    "B": 0.5
        }
exp_with_p0 = T2Ramsey(qubit, delays, unit=unit)
exp_with_p0.set_analysis_options(user_p0=user_p0, plot=True)
expdata_with_p0 = exp_with_p0.run(backend=backend, shots=2000)
print(expdata_with_p0)

The units can be changed, but the output in the result is always given in seconds. The units in the backend must be adjusted accordingly.

In [ ]:
from qiskit.utils import apply_prefix
unit = 'ns'
delays = list(range(1000, 150000, 2000))
conversion_factor = apply_prefix(1, unit)
print(conversion_factor)

In [ ]:
p0={"a_guess":[0.5], "t2ramsey":[80000], "f_guess":[0.00002], "phi_guess":[0.0],
                        "b_guess": [0.5]}
backend_in_ns = T2RamseyBackend(
                    p0=p0,
                    initial_prob_plus=[0.0],
                    readout0to1=[0.02],
                    readout1to0=[0.02],
                    conversion_factor=conversion_factor
                )
exp_in_ns = T2Ramsey(qubit, delays, unit=unit)
exp_in_ns.set_analysis_options(user_p0=None, plot=True)
expdata_in_ns = exp_in_ns.run(backend=backend_in_ns, shots=2000)
print(expdata_in_ns)

It is possible to add data to an experiment, after the analysis of the first set of data. In the next example we add exp2 to `exp_in_ns` that we showed above.

In [ ]:
#import numpy as np

more_delays = list(range(2000, 150000, 2000))         
exp_new = T2Ramsey(qubit, more_delays, unit=unit)
exp_new.set_analysis_options(user_p0=None, plot=True)
expdata_new = exp_new.run(
            backend=backend_in_ns,
            experiment_data=expdata_in_ns,
            shots=2000
        )
result_new = expdata_new.analysis_result(1)
print(result_new)